# Curve regression with TensorFlow 2

This ntoebook provides a simple example of performing a regression problem with TensorFlow 2. The notebook could be adapted by changing the fitting functions to address a wide variety of regression tasks.

#### First, import numpy, matplotlib, and tensorflow

Also, check the version of TF.

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
print(tf.__version__)

In [ ]:
%matplotlib inline
plt.style.use("astr19_matplotlib_style.txt")

#### Second, create a function that will generate some data to fit

In [ ]:
#reference function to create data, and make the fit
#first argument is the variable x
#other arguments are amplitude, frequency, and phase
def sinusoid(x,A,f,p):
    return A*np.sin(2.0*np.pi*f*x+p)

In [ ]:
#function that will generate some data
def generate_data(xmin=0.0,xmax=10.0,A=1.0,f=0.5,p=1.0,s=0.25,n=100):
    #xmin is the minimum range of the data
    #xmax is the maximum range of the data
    #A is the Amplitude
    #f is the frequency
    #p is the phase
    #s is the gaussian stdev
    #n is the number of samples

    #create n points randomly (uniformly) distributed between xmin and xmax
    x = np.random.uniform(low=xmin,high=xmax,size=n)
    x = np.asarray(x,dtype=np.float32) #convert to a float
    
    #y have a value centered on the real curve, but with gaussian error
    y = sinusoid(x,A,f,p) + s*np.random.randn(n)
    y = np.asarray(y,dtype=np.float32)

    y_err = np.full(n,s,dtype=np.float32)

    #return x, y, and y_err values
    return x,y,y_err

#### Generate the data

In [ ]:
x, y, y_err = generate_data(s=0.1)

#### Plot the data

In [ ]:
f,ax = plt.subplots(1,1,figsize=(7,7))
ax.errorbar(x,y,y_err,fmt='o',label='Data')
ax.set_xlim([-0.1,10.1])
ax.set_ylim([-2.,2.])
ax.text(0.3,1.75,'Sigma = 1',color='0',fontsize=12)
ax.set_xlabel('x',fontsize=20)
ax.set_ylabel('y',fontsize=20)
plt.legend(frameon=True,fontsize=20,handletextpad=0)
plt.show()

#### Let's prepare our fitted model

First, we declare our fitted variables as global tf.Variables

In [ ]:
#Amplitude
A_init = 3.0
A_fit = tf.Variable(A_init, name='amplitude')

#frequency
f_init = 0.5
f_fit = tf.Variable(f_init, name='frequency')

#phase
p_init = 2.0
p_fit = tf.Variable(p_init,name='phase')

#### Now, let's define the model for tensorflow to fit

The function needs to be decorated with the @tf.function decorator.

In [ ]:
#define sinusoidal model
@tf.function
def sinusoidal_model(x):
    pi = tf.math.acos(-1.0)
    return A_fit*tf.math.sin(2.0*pi*f_fit*x + p_fit)

#### We need to define the loss function, using tensorflow

We will use mean-squared error

In [ ]:
#define loss function
@tf.function
def mse(y_true, y_pred):
    #mse between true and predicted values
    return tf.losses.mse(y_true,y_pred)

#### We also need to define the optimizer that tells us how to do gradient descent

We'll use the pre-defined Adam optimizer from Keras.

In [ ]:
#Adam Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

#### Now we train the model iteratively

In [ ]:
#start training
epochs = 5000 #number of epochs to run
display_step = 1000 #print info every this number of steps

A_array = np.zeros(epochs)
f_array = np.zeros(epochs)
p_array = np.zeros(epochs)
l_array = np.zeros(epochs)

for epoch in range(epochs):

    #monitor training / display steps
    if((epoch%display_step)==0):
        pred = sinusoidal_model(x)
        loss = mse(pred,y)

        #to print this info, we need to convert to numpy arrays
        print(f"Epoch {epoch} : Loss {loss.numpy()}, A: {A_fit.numpy()} f: {f_fit.numpy()} p:{p_fit.numpy()}")

    A_array[epoch] = A_fit.numpy()
    f_array[epoch] = f_fit.numpy()
    p_array[epoch] = p_fit.numpy()
    l_array[epoch] = loss.numpy()
    #In TF2, gradients are handled using
    #the GradientTape class.
    #begin GradientTape and optimize
    with tf.GradientTape() as g:
        pred = sinusoidal_model(x)
        loss = mse(y,pred)

    #compute dm, db, the gradients
    #with respect to (A,f,p)_fit
    gradients = g.gradient(loss, [A_fit,f_fit,p_fit])

    #update (A,f,p)_fit with the gradients x learning
    optimizer.apply_gradients(zip(gradients,[A_fit,f_fit,p_fit]))

#print the last answer
print(f"Epoch {epoch} : Loss {loss.numpy()}, A: {A_fit.numpy()} f: {f_fit.numpy()} p:{p_fit.numpy()}")
print("Done!")

#### Let's plot the data and the model

In [ ]:
x_model = np.linspace(0,10,1000)
y_model = sinusoid(x_model,A_fit,f_fit,p_fit)

In [ ]:
f,ax = plt.subplots(1,1,figsize=(7,7))
ax.errorbar(x,y,y_err,fmt='o',zorder=5,label='Data')
ax.plot(x_model,y_model,color='magenta',zorder=6,label='Best-fit Model')
ax.set_xlim([-0.1,10.1])
ax.set_ylim([-2.,2.])
ax.text(0.3,1.75,'Sigma = 1',color='0',fontsize=12)
ax.set_xlabel('x',fontsize=20)
ax.set_ylabel('y',fontsize=20)
plt.legend(frameon=True,fontsize=10,handletextpad=1)
plt.show()

In [ ]:
plt.plot(A_array,label='A')
plt.plot(f_array,label='f')
plt.plot(p_array,label='p')
plt.plot(l_array,label='loss')
plt.legend()
plt.show()

In [ ]:
from ipywidgets import interactive

def sin_epoch(x, epoch):
    a = A_array[epoch]
    f = f_array[epoch]
    p = p_array[epoch]
    return a*np.sin(2.0*np.pi *f * x + p)

# Create interactive plot
def interactive_plot(epoch):
    plt.figure(figsize=(8, 6))
    plt.errorbar(x,y,y_err,fmt='o',zorder=5,label='Data')
    plt.plot(x_model, sin_epoch(x_model, epoch), color='magenta',zorder=6,label='Best-fit Model')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.xlim([-0.1,10.1])
    plt.ylim([-2.,2.])
    plt.legend()
    plt.grid(True)
    plt.show()

# Create slider
p_slider = interactive(interactive_plot, epoch=(0, 4999, 1),continuous_update=False)

# Set default value of slider to 0
p_slider.children[0].value = 0

output = p_slider.children[-1]
output.layout.height = '450px'

# Display the slider
display(p_slider)